In [2]:
#import needed packages
import os
from pathlib import Path

import earthpy
import geopandas as gpd
import pandas as pd

# Get month names
import calendar

# Libraries for Dynamic mapping
import cartopy.crs as ccrs
import hvplot.pandas
import panel as pn

#more packages needed
import json
import requests
import geopandas as gpd
from io import BytesIO 

import time
import zipfile
from getpass import getpass
from glob import glob
import shutil

import pygbif.occurrences as occ
import pygbif.species as species
import requests

In [3]:
# Create data directory
project = earthpy.Project(
    dirname='bigcat_migration_data')

#Turn string into object
project_dir = Path(project.project_dir)

#create folder for data
project_dir.mkdir(parents=True, exist_ok=True)

# Display the project directory
project.project_dir

PosixPath('/workspaces/data/bigcat_migration_data')

In [4]:
# GBIF needs a username, password, and email 
# All 3 need to match the account
reset = False

# Request and store username
if (not ('GBIF_USER'  in os.environ)) or reset:
    os.environ['GBIF_USER'] = input('GBIF username:')

# Securely request and store password
if (not ('GBIF_PWD'  in os.environ)) or reset:
    os.environ['GBIF_PWD'] = getpass('GBIF password:')
    
# Request and store account email address
if (not ('GBIF_EMAIL'  in os.environ)) or reset:
    os.environ['GBIF_EMAIL'] = input('GBIF email:')

In [5]:
#load the species data for selected species
backbone = species.name_backbone(name='Puma concolor')

#save the unique identifier 
species_key = backbone['usageKey']


In [6]:
# Only download once
### set file name for download
gbif_pattern = os.path.join(project_dir, '*.csv')

### double check that there isn't already a file that matches this pattern.
### if it already exists, skip the whole conditional
### and go straight to the line: gbif_path = glob(gbif_pattern)[0]
if not glob(gbif_pattern):

    ### only submit a download request to GBIF once
    ### if GBIF_DOWNLOAD_KEY is not defined in our environment, make the download request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:

        ### submit a query to GBIF
        gbif_query = occ.download([

            ### add your species key here
            f"taxonKey = {species_key}",

            ### filter out results that are missing coordinates
            "hasCoordinate = True",

            ### choose a year to include
            "year = 2023",
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']

    ### use the occurrence command module in pygbif to get the metadata
    wait = occ.download_meta(download_key)['status']

    ### check if the status of the download = "SUCCEEDED"
    ### wait and loop through until it finishes
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']

        ### don't want to re-query the API in the loop too frequently
        time.sleep(5)

    # Download GBIF data when it's ready
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=project_dir)

    # Unzip GBIF data using the zipfile package
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=project_dir)

# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]

INFO:Your download key is 0005625-251025141854904
INFO:Download file size: 99832 bytes
INFO:On disk at /workspaces/data/bigcat_migration_data/0005625-251025141854904.zip


In [7]:
# Give the download a descriptive name
gbif_path = project.project_dir / 'taxon_gbif'

#Find and name path of recent download
original_gbif_path = Path('/workspaces/data/bigcat_migration_data'
'/0005625-251025141854904.csv')

# Move file to descriptive path
shutil.move(original_gbif_path, gbif_path)

PosixPath('/workspaces/data/bigcat_migration_data/taxon_gbif')

In [8]:
# Load the GBIF data as dataframe
gbif_df = pd.read_csv(
    gbif_path,
    delimiter='\t',
    index_col='gbifID',
    usecols=['gbifID','month','decimalLatitude','decimalLongitude']
)

# Convert data to GDF
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df, 
        geometry=gpd.points_from_xy(
            gbif_df.decimalLongitude, 
            gbif_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month','geometry']]
)

# Check results
gbif_gdf.total_bounds

array([-128.604153,  -56.459571,   83.61557 ,   56.116539])

In [9]:
# Merge the GBIF observations into a single geometry
gbif_union = gbif_gdf.geometry.union_all().envelope

# Convert geometry to geoJSON
gbif_geojson = gbif_union.__geo_interface__

gbif_geojson


{'type': 'Polygon',
 'coordinates': (((-128.604153, -56.459571),
   (83.61557, -56.459571),
   (83.61557, 56.116539),
   (-128.604153, 56.116539),
   (-128.604153, -56.459571)),)}

In [10]:
# Construct ArcGIS-compatible JSON
arcgis_geom = json.dumps(dict(
    rings=gbif_geojson["coordinates"],
    spatialReference={"wkid": 4326}
))

In [11]:
# Prepare API request
eco_url = (
    "https://services5.arcgis.com/0AFsQflykfA9lXZn"
    "/ArcGIS/rest/services"
    "/WWF_Terrestrial_Ecoregions_Of_The_World_official_teow"
    "/FeatureServer/0/query")
eco_params = {
    "f": "geojson",
    "where": "1=1",
    "outFields": "eco_code,area_km2",
    "returnGeometry": "true",
    # Return polygons containing any GBIF observation
    "spatialRel": "esriSpatialRelIntersects",  
    "geometryType": "esriGeometryPolygon",
    # Override web Mercator server default
    "inSR": "4326",
    "outSR": "4326",
    # Must format geometry
    "geometry": arcgis_geom
}

# Submit API request
eco_resp = requests.get(
    eco_url, params=eco_params,
    headers={"Accept-Encoding": "identity"})
eco_resp.raise_for_status()

# Load binary data to DataFrame
eco_gdf = gpd.read_file(BytesIO(eco_resp.content))

# Check the download
eco_gdf.head()

,eco_code,area_km2,geometry
0,NT0404,248398,"POLYGON ((-74.32428 -45.42114, -74.33501 -45.4..."
1,NT0404,248398,"POLYGON ((-74.1213 -44.09145, -74.12397 -44.08..."
2,AT1012,19515,"POLYGON ((30.7211 -29.71616, 30.71552 -29.7090..."
3,NT0702,125589,"POLYGON ((-67.98364 -13.79293, -67.98972 -13.7..."
4,NT0128,114506,"POLYGON ((-72.72287 -3.54497, -72.724 -3.5443,..."


In [12]:
# Save the ecoregion data
eco_dir = project.project_dir / 'ecoregions'
eco_dir.mkdir(exist_ok=True)
eco_path = eco_dir / 'ecoregions.shp'
eco_gdf.to_file(eco_path)

INFO:Created 2,000 records


In [13]:
# Simplify the geometry to speed up processing
eco_gdf.geometry = eco_gdf.simplify(.1, preserve_topology=False)

# Change the CRS to Mercator for mapping
eco_gdf = eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

:Polygons   [Longitude,Latitude]

In [14]:
# Reproject GBIF points to Mercator to match ecoregions
gbif_gdf_merc = gbif_gdf.to_crs(ccrs.Mercator())

# Plot ecoregions as polygons
map_plot = eco_gdf.hvplot(geo=True, crs=ccrs.Mercator(), alpha=0.5, legend=False)

# Overlay GBIF points as red dots
map_plot = map_plot * gbif_gdf_merc.hvplot(geo=True, crs=ccrs.Mercator(), color='red', size=5)

# Display the map
map_plot

:Overlay
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]

In [15]:
# See if both datasets use the same CRS 
eco_gdf_crs = eco_gdf.to_crs(gbif_gdf.crs)

# Spatial join: find which ecoregion each GBIF observation falls in
gbif_ecoregion_gdf = (
    gpd.sjoin(
        gbif_gdf,
        eco_gdf_crs,
        how='inner',
        predicate='within' 
    )
    # Select key columns
    [['eco_code', 'month']]
    .rename(columns={
        'eco_code': 'ecoregion'
    })
)

# add GBIF ID back in for reference
gbif_ecoregion_gdf['observation_ID'] = gbif_ecoregion_gdf.index


occurrence_df = (
    gbif_ecoregion_gdf
    # For each ecoregion, for each month...
    .groupby(['ecoregion', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('observation_ID', 'count'))
    .reset_index()
)

#Get rid of rare observations (possible misidentifications)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]

# Take the mean by ecoregion
mean_occurrences_ecoregion = (
   occurrence_df
   .groupby('ecoregion')['occurrences']
   .mean()
)

#Take the mean by month
mean_occurrences_month = (
    occurrence_df
    .groupby('month')['occurrences']
    .mean()
)

# Normalize by ecoregion and month for sampling effort
occurrence_df = (
    occurrence_df
    .assign(
        norm_occurrences= lambda x:
        x['occurrences']
        / x['ecoregion'].map(mean_occurrences_ecoregion)
        / x['month'].map(mean_occurrences_month)
    )
)


In [16]:
#reset index of occurrence df
occurrence_df = occurrence_df.reset_index()

#rename column of eco gdf
eco_gdf = eco_gdf.rename(columns={'eco_code':'ecoregion'})

# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = eco_gdf.merge(
    occurrence_df[['month','ecoregion','norm_occurrences']],
    on='ecoregion',
    how='inner',
).set_index('ecoregion')

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = eco_gdf.to_crs(ccrs.Mercator()).total_bounds

month_widget = pn.widgets.DiscreteSlider(
    options={
        calendar.month_name[month_num]: month_num
        for month_num in range(1,12)
    }
)

# Plot occurrence by ecoregion and month
migration_plot = (
    occurrence_gdf
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Mountain Lion Occurrences by Ecoregion and Month",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widgets={'month':month_widget},
        widget_location='bottom'
    )
)

# Save the plot
migration_plot.save('migration.html', embed=True)

migration_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'170f42bd-3c5e-4dda-ab81-80652cc8264e': {'version…